In [1]:
import cv2
import numpy as np
import os
import json

# 설정
input_images_folder = '/root/eliceAI/mm_jh/mmdetection/EDA/Train_for_long_crack'  # 원본 이미지 폴더
input_json_path = '/root/eliceAI/mm_jh/mmdetection/EDA/LongCrack_annotations/filtered_annotations.json'  # COCO 포맷 JSON 파일 경로
output_images_folder = '/root/eliceAI/mm_jh/mmdetection/EDA/RotationIMG_long_crack'  # 회전된 이미지를 저장할 폴더
output_json_path = '/root/eliceAI/mm_jh/mmdetection/EDA/RotationIMG_long_crack/LongCrack_rotation.json'  # 회전된 바운딩 박스를 저장할 JSON 파일 경로

# JSON 파일 로딩
with open(input_json_path, 'r') as f:
    coco_data = json.load(f)

# 폴더 생성
os.makedirs(output_images_folder, exist_ok=True)

def rotate_image(image_path, output_path):
    img = cv2.imread(image_path)
    if img is None:
        return None, None
    
    # 이미지 크기
    (h, w) = img.shape[:2]
    
    # 이미지 회전
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, 90, 1.0)
    rotated_img = cv2.warpAffine(img, matrix, (w, h))
    
    cv2.imwrite(output_path, rotated_img)
    
    return img, rotated_img

def rotate_bbox(bbox, img_shape, angle):
    (h, w) = img_shape[:2]
    if angle == 90:
        x, y, w, h = bbox
        return [y, w - x, h - (y + h), w]
    elif angle == 180:
        x, y, w, h = bbox
        return [w - x, h - y, x + w, y + h]
    elif angle == 270:
        x, y, w, h = bbox
        return [h - y, x, w - x, h - (y + h)]

def update_annotations(annotations, image_id, angle):
    updated_annotations = []
    for ann in annotations:
        if ann['image_id'] == image_id:
            bbox = ann['bbox']
            updated_bbox = rotate_bbox(bbox, (img.shape[0], img.shape[1]), angle)
            ann['bbox'] = updated_bbox
            updated_annotations.append(ann)
        else:
            updated_annotations.append(ann)
    return updated_annotations

# 이미지 및 바운딩 박스 회전
rotated_annotations = []
for img_info in coco_data['images']:
    image_id = img_info['id']
    image_file = img_info['file_name']
    
    image_path = os.path.join(input_images_folder, image_file)
    output_image_path = os.path.join(output_images_folder, image_file)
    
    img, rotated_img = rotate_image(image_path, output_image_path)
    if rotated_img is not None:
        rotated_annotations.extend(update_annotations(coco_data['annotations'], image_id, 90))

# 회전된 바운딩 박스를 포함한 JSON 저장
with open(output_json_path, 'w') as f:
    json.dump({
        'images': coco_data['images'],
        'annotations': rotated_annotations,
        'categories': coco_data['categories']
    }, f, indent=4)

print("Rotation complete.")


OSError: [Errno 28] No space left on device